In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from functions import *

pd.set_option('display.max_columns', None)

df_merged = pd.read_csv("data/df_merged_national.csv")

In [3]:
df_merged = clean_df(df_merged)
df_merged = every_float_to_int(df_merged)

In [4]:
df_no_provincias = df_merged[(df_merged["region"] == "    DESCONOCIDA") | (df_merged["region"] == "    EN EL EXTRANJERO") | (df_merged["region"] == "    TOTAL NACIONAL")]

df_no_provincias.to_csv("data/df_no_provincias.csv", index=False)

In [5]:
df_merged = df_merged[~df_merged.index.isin(df_no_provincias.index)]

In [6]:
df_num = df_merged.iloc[:, df_merged.columns != 'region']

In [7]:
df_merged = add_coordinates_from_dict(df_merged)

In [8]:
regions = {
    "ANDALUCÍA": 0,
    "ARAGÓN": 1,
    "ASTURIAS (PRINCIPADO DE)": 2,
    "BALEARS (ILLES)": 3,
    "CANTABRIA": 4,
    "CANARIAS": 5,
    "CASTILLA - LA MANCHA": 6,
    "CASTILLA Y LEÓN": 7,
    "CATALUÑA": 8,
    "CIUDAD AUTÓNOMA DE CEUTA": 9,
    "CIUDAD AUTÓNOMA DE MELILLA": 10,
    "COMUNITAT VALENCIANA": 11,
    "EXTREMADURA": 12,
    "GALICIA": 13,
    "MADRID (COMUNIDAD DE)": 14,
    "MURCIA (REGIÓN DE)": 15,
    "NAVARRA (COMUNIDAD FORAL DE)": 16,
    "PAÍS VASCO": 17,
    "RIOJA (LA)": 18
}


df_merged["region"] = df_merged["region"].replace(regions)


C:\Users\VSPC\AppData\Local\Temp\ipykernel_23456\2512969448.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merged["region"] = df_merged["region"].replace(regions)


In [9]:
df_merged.insert(0, "year", df_merged.pop("year"))

In [10]:
df_merged["avg_crimes"] = df_merged.iloc[:, 2:-4].mean(axis=1)      # [:, 2:-2] quiero -> todas las filas, desde la 2da hasta la penultima columna

In [11]:
df_merged = delete_sub_crimes(df_merged)

In [12]:
df_merged["years_from_today"] = 2024 - df_merged["year"]

In [13]:
df_poblacion = pd.read_csv("data/cleaned_pop.csv")


df_poblacion.drop(["Nacional", "Sexo"], inplace=True, axis=1)

regions_pop_version = {
    "01 Andalucía": 0,
    "02 Aragón": 1,
    "03 Asturias Principado de": 2,
    "04 Balears Illes": 3,
    "06 Cantabria": 4,
    "05 Canarias": 5,
    "08 Castilla - La Mancha": 6,
    "07 Castilla y León": 7,
    "09 Cataluña": 8,
    "18 Ceuta": 9,
    "19 Melilla": 10,
    "10 Comunitat Valenciana": 11,
    "11 Extremadura": 12,
    "12 Galicia": 13,
    "13 Madrid Comunidad de": 14,
    "14 Murcia Región de": 15,
    "15 Navarra Comunidad Foral de": 16,
    "16 País Vasco": 17,
    "17 Rioja La": 18
}


df_poblacion["region"] = df_poblacion["region"].replace(regions_pop_version)

C:\Users\VSPC\AppData\Local\Temp\ipykernel_23456\1471067258.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_poblacion["region"] = df_poblacion["region"].replace(regions_pop_version)


In [14]:
df_poblacion

,region,year,Total
0,0,2019,8.414.240
1,0,2018,8.384.408
2,0,2017,8.379.820
3,0,2016,8.388.107
4,0,2015,8.399.043
...,...,...,...
185,10,2014,84.509
186,10,2013,83.679
187,10,2012,80.802
188,10,2011,78.476


In [15]:
df_merged = df_merged.merge(df_poblacion, on=["region", "year"])

df_merged.rename(columns={"Total": "population"}, inplace=True)

In [16]:
df_merged.columns

Index(['year', 'region', '1_against_people', '2_against_freedom',
       '3_sexual_freedom', '4_family_relationships', '5_property_crimes',
       '6_collective_security', '7_forgeries', '8_public_administration',
       '9_justice_administration', 'latitude', 'longitude', 'avg_crimes',
       'years_from_today', 'population'],
      dtype='object')

In [17]:
crime_values = ['1_against_people', '2_against_freedom',
       '3_sexual_freedom', '4_family_relationships', '5_property_crimes',
       '6_collective_security', '7_forgeries', '8_public_administration',
       '9_justice_administration']

In [18]:
df_unpivot = pd.melt(df_merged, id_vars=['year', 'region', 'latitude', 'longitude', 'population'], value_vars=crime_values)

df_unpivot

,year,region,latitude,longitude,population,variable,value
0,2019,0,37.3873,-5.9869,8.414.240,1_against_people,37130
1,2019,1,41.6488,-0.8891,1.319.291,1_against_people,4760
2,2019,2,43.3619,-5.8494,1.022.800,1_against_people,3368
3,2019,3,39.5712,2.6466,1.149.460,1_against_people,8110
4,2019,5,28.2916,-16.6291,2.153.389,1_against_people,12396
...,...,...,...,...,...,...,...
1705,2010,16,42.6954,-1.6761,636.924,9_justice_administration,235
1706,2010,17,43.2630,-2.9349,2.178.339,9_justice_administration,1170
1707,2010,18,42.2871,-2.5396,322.415,9_justice_administration,137
1708,2010,9,35.8894,-5.3198,80.579,9_justice_administration,101


In [19]:
df_unpivot_transformed = df_unpivot.copy()

for crime in crime_values:
    column_name = f'{crime}_indicator'
    df_unpivot_transformed[column_name] = (df_unpivot_transformed['variable'] == crime).astype(int)

df_unpivot_transformed

,year,region,latitude,longitude,population,variable,value,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator
0,2019,0,37.3873,-5.9869,8.414.240,1_against_people,37130,1,0,0,0,0,0,0,0,0
1,2019,1,41.6488,-0.8891,1.319.291,1_against_people,4760,1,0,0,0,0,0,0,0,0
2,2019,2,43.3619,-5.8494,1.022.800,1_against_people,3368,1,0,0,0,0,0,0,0,0
3,2019,3,39.5712,2.6466,1.149.460,1_against_people,8110,1,0,0,0,0,0,0,0,0
4,2019,5,28.2916,-16.6291,2.153.389,1_against_people,12396,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,2010,16,42.6954,-1.6761,636.924,9_justice_administration,235,0,0,0,0,0,0,0,0,1
1706,2010,17,43.2630,-2.9349,2.178.339,9_justice_administration,1170,0,0,0,0,0,0,0,0,1
1707,2010,18,42.2871,-2.5396,322.415,9_justice_administration,137,0,0,0,0,0,0,0,0,1
1708,2010,9,35.8894,-5.3198,80.579,9_justice_administration,101,0,0,0,0,0,0,0,0,1


In [20]:
df_values = get_previous_years(df_unpivot_transformed)

df_values 

,year,region,latitude,longitude,population,values_year,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator,values_year_minus_1,values_year_minus_2,values_year_minus_3
0,2019,0,37.3873,-5.9869,8.414.240,37130,1,0,0,0,0,0,0,0,0,35043.0,35087.0,33661.0
1,2019,1,41.6488,-0.8891,1.319.291,4760,1,0,0,0,0,0,0,0,0,4474.0,4146.0,4005.0
2,2019,2,43.3619,-5.8494,1.022.800,3368,1,0,0,0,0,0,0,0,0,3159.0,3160.0,3239.0
3,2019,3,39.5712,2.6466,1.149.460,8110,1,0,0,0,0,0,0,0,0,7545.0,7096.0,6609.0
4,2019,5,28.2916,-16.6291,2.153.389,12396,1,0,0,0,0,0,0,0,0,11805.0,11744.0,11021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,2010,16,42.6954,-1.6761,636.924,235,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN
1706,2010,17,43.2630,-2.9349,2.178.339,1170,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN
1707,2010,18,42.2871,-2.5396,322.415,137,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN
1708,2010,9,35.8894,-5.3198,80.579,101,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN


In [21]:
df_total_values = df_values.dropna(subset=['values_year_minus_1', 'values_year_minus_2', 'values_year_minus_3']).reset_index(drop=True)

df_total_values

,year,region,latitude,longitude,population,values_year,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator,values_year_minus_1,values_year_minus_2,values_year_minus_3
0,2019,0,37.3873,-5.9869,8.414.240,37130,1,0,0,0,0,0,0,0,0,35043.0,35087.0,33661.0
1,2019,1,41.6488,-0.8891,1.319.291,4760,1,0,0,0,0,0,0,0,0,4474.0,4146.0,4005.0
2,2019,2,43.3619,-5.8494,1.022.800,3368,1,0,0,0,0,0,0,0,0,3159.0,3160.0,3239.0
3,2019,3,39.5712,2.6466,1.149.460,8110,1,0,0,0,0,0,0,0,0,7545.0,7096.0,6609.0
4,2019,5,28.2916,-16.6291,2.153.389,12396,1,0,0,0,0,0,0,0,0,11805.0,11744.0,11021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,2013,16,42.6954,-1.6761,644.477,198,0,0,0,0,0,0,0,0,1,234.0,230.0,235.0
1193,2013,17,43.2630,-2.9349,2.191.682,1470,0,0,0,0,0,0,0,0,1,1458.0,1357.0,1170.0
1194,2013,18,42.2871,-2.5396,322.027,115,0,0,0,0,0,0,0,0,1,147.0,141.0,137.0
1195,2013,9,35.8894,-5.3198,84.180,125,0,0,0,0,0,0,0,0,1,157.0,84.0,101.0


In [22]:
# Tabla final

#df_total_values.to_csv('df_total_values.csv', index=False)

df_total_values['population'] = df_total_values['population'].str.replace('.', '', regex=False).astype(float)




### Aplicar PCA para las columnas

In [23]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

columns_pca = ['values_year_minus_1', 'values_year_minus_2', 'values_year_minus_3']
values_year_data = df_total_values[columns_pca]

scaler = StandardScaler()
values_year_scaled = scaler.fit_transform(values_year_data)

pca = PCA(n_components=0.9, random_state=42)
pca_result = pca.fit_transform(values_year_scaled)

pca_column = [f'Value_year_PCA{i+1}' for i in range(pca_result.shape[1])]
df_pca_values = pd.DataFrame(pca_result, columns=pca_column)

df_pca_values

,Value_year_PCA1
0,0.862743
1,-0.305943
2,-0.345329
3,-0.195319
4,-0.024596
...,...
1192,-0.458865
1193,-0.416703
1194,-0.462375
1195,-0.463435


In [24]:
# Merge

df_total_pca_values = df_total_values.copy()

df_total_pca_values['Values_year_PCA'] = df_pca_values['Value_year_PCA1']

df_total_pca_values = df_total_pca_values.drop(columns=columns_pca)

df_total_pca_values.to_csv('df_total_pca_values.csv', index=False)

In [25]:
df_total_values_201017 = df_total_pca_values[df_total_values['year'] <= 2017]

df_total_values_201819 = df_total_pca_values[df_total_values['year'] >= 2018]

In [26]:
features = [
    'year','latitude', 'longitude', 'population',
    '1_against_people_indicator', '2_against_freedom_indicator',
    '3_sexual_freedom_indicator', '4_family_relationships_indicator',
    '5_property_crimes_indicator', '6_collective_security_indicator',
    '7_forgeries_indicator', '8_public_administration_indicator',
    '9_justice_administration_indicator', 'Values_year_PCA'
    ]

X_train = df_total_values_201017[features]
y_train = df_total_values_201017['values_year']

X_test = df_total_values_201819[df_total_values_201819['year'] == 2018][features]
y_test = df_total_values_201819[df_total_values_201819['year'] == 2018]['values_year']

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((855, 14), (855,), (171, 14), (171,))

In [27]:
df_total_values_201017[features]

,year,latitude,longitude,population,1_against_people_indicator,2_against_freedom_indicator,3_sexual_freedom_indicator,4_family_relationships_indicator,5_property_crimes_indicator,6_collective_security_indicator,7_forgeries_indicator,8_public_administration_indicator,9_justice_administration_indicator,Values_year_PCA
38,2017,37.3873,-5.9869,8379820.0,1,0,0,0,0,0,0,0,0,1.333843
39,2017,41.6488,-0.8891,1308750.0,1,0,0,0,0,0,0,0,0,-0.279516
40,2017,43.3619,-5.8494,1034960.0,1,0,0,0,0,0,0,0,0,-0.302724
41,2017,39.5712,2.6466,1115999.0,1,0,0,0,0,0,0,0,0,-0.164682
42,2017,28.2916,-16.6291,2108121.0,1,0,0,0,0,0,0,0,0,0.122110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,2013,42.6954,-1.6761,644477.0,0,0,0,0,0,0,0,0,1,-0.458865
1193,2013,43.2630,-2.9349,2191682.0,0,0,0,0,0,0,0,0,1,-0.416703
1194,2013,42.2871,-2.5396,322027.0,0,0,0,0,0,0,0,0,1,-0.462375
1195,2013,35.8894,-5.3198,84180.0,0,0,0,0,0,0,0,0,1,-0.463435


### Random forest sin estandarizar y normalizar

In [28]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

In [35]:
rf_reg = RandomForestRegressor(
    n_estimators=300, 
    max_leaf_nodes=None,
    min_samples_split=5,
    n_jobs=-1, 
    random_state=42
    )

rf_reg.fit(X_train, y_train)
y_pred = rf_reg.predict(X_test)



array([3.63015381e+04, 3.58328453e+03, 2.89474851e+03, 6.40251695e+03,
       9.90328594e+03, 2.20538789e+03, 6.93234342e+03, 6.04618796e+03,
       1.97860050e+04, 1.98805766e+04, 2.95710549e+03, 7.28662935e+03,
       2.20035589e+04, 6.01986427e+03, 2.33810090e+03, 6.00624600e+03,
       9.29688066e+02, 6.89260223e+02, 9.10380294e+02, 2.04438585e+04,
       4.23637025e+03, 2.25275662e+03, 5.85001201e+03, 1.00183282e+04,
       1.83541303e+03, 6.28485441e+03, 5.83479561e+03, 1.95504067e+04,
       1.41081925e+04, 4.32613168e+03, 7.81699543e+03, 1.50927808e+04,
       4.37136300e+03, 2.23065359e+03, 9.41829811e+03, 5.32531139e+02,
       3.06975806e+02, 3.34881155e+02, 2.10183387e+03, 2.92146038e+02,
       1.56986633e+02, 5.35799666e+02, 6.15834398e+02, 6.86985210e+01,
       3.60764835e+02, 3.63387536e+02, 2.05764900e+03, 1.41780154e+03,
       1.60648028e+02, 4.95335632e+02, 1.71512978e+03, 4.02681114e+02,
       2.49716101e+02, 4.62937789e+02, 4.58300578e+01, 2.82930576e+01,
      

In [30]:
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")

Mean Absolute Error (MAE): 1033.0388923532062
Root Mean Squared Error (RMSE): 4707.907926358282
R² Score: 0.9896681902859178


### Estcalado

In [31]:
features_nn = [
    'year','latitude', 'longitude', 'population',
    '1_against_people_indicator', '2_against_freedom_indicator',
    '3_sexual_freedom_indicator', '4_family_relationships_indicator',
    '5_property_crimes_indicator', '6_collective_security_indicator',
    '7_forgeries_indicator', '8_public_administration_indicator',
    '9_justice_administration_indicator', 'Values_year_PCA'
    ]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[features_nn])
X_test_scaled = scaler.transform(X_test[features_nn])

rf_reg_nn = RandomForestRegressor(
    n_estimators=300, 
    max_leaf_nodes=None,
    min_samples_split=5,
    n_jobs=-1, 
    random_state=42
)

rf_reg_nn.fit(X_train_scaled, y_train)
y_pred = rf_reg_nn.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred) 
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:}")
print(f"Root Mean Squared Error (RMSE): {rmse:}")
print(f"R² Score: {r2:}")

Mean Absolute Error (MAE): 1035.459572089866
Root Mean Squared Error (RMSE): 4709.648084016563
R² Score: 0.9896605510963612


### For loop

In [40]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

X_train = df_total_values_201017[features]
y_train = df_total_values_201017['values_year']

X_test = df_total_values_201819[df_total_values_201819['year'] == 2018][features]
y_test = df_total_values_201819[df_total_values_201819['year'] == 2018]['values_year']

rf_reg = RandomForestRegressor(
    n_estimators=300, 
    max_leaf_nodes=None,
    min_samples_split=5,
    n_jobs=-1, 
    random_state=42
    )

rf_reg.fit(X_train, y_train)

mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")


Mean Absolute Error (MAE): 1033.0388923532068
Root Mean Squared Error (RMSE): 4707.90792635829
R² Score: 0.9896681902859178


In [ ]:
future_years = [2018, 2019, 2020, 2021, 2022, 2023, 2024]

df_try_1 = df_total_values_201017.copy()

for year in future_years:
    X_year = df_try_1[df_try_1['year'] == year - 1][features_nn]

    y_pred_year = rf_reg.predict(X_year)

    new_year_df = df_try_1[df_try_1['year']==year-1].copy()
    new_year_df['year'] = year
    new_year_df['values_year'] = y_pred_year

    df_try_1 = pd.concat([df_try_1, new_year_df], ignore_index=True)

    X_train_new_year = df_try_1[df_try_1['year'] <= year][features_nn]
    y_train_new_year = df_try_1[df_try_1['year'] <= year]['values_year']

    rf_reg.fit(X_train_new_year, y_train_new_year)

df_try_1